In [1]:
import joblib
import os
from numpy.linalg import inv
from API.utils import *
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

In [2]:
os.listdir('./model')

['ANN(58_to_sp162).pkl',
 'ANN(energy).pkl',
 'ANN(tray).pkl',
 'eva.pkl',
 'OperationOptimModel.pkl',
 'SVR(4_to_23).pkl',
 'transformer(23_to_54).pkl',
 'transformer(33_to_15).pkl',
 'transformer(43_to_33).pkl',
 'transformer(54_to_23).pkl',
 'transformer(54_to_33).pkl',
 'transformer(SP162_to_Y162).pkl']

In [3]:
os.listdir('./data/phase_2/cleaned')

['col_names.pkl', 'train(energy).csv', 'train.csv']

# load pretrained model

In [4]:
A = joblib.load('./model/SVR(4_to_23).pkl')
B = joblib.load('./model/transformer(23_to_54).pkl')
C = joblib.load('./model/ANN(58_to_sp162).pkl')
D = joblib.load('./model/transformer(SP162_to_Y162).pkl')
E = joblib.load('./model/ANN(energy).pkl')
F = joblib.load('./model/transformer(54_to_33).pkl')
G = joblib.load('./model/transformer(33_to_15).pkl')
H = joblib.load('./model/transformer(43_to_33).pkl')
I = joblib.load('./model/transformer(54_to_23).pkl')
col_names = joblib.load('./data/phase_2/cleaned/col_names.pkl')

In [5]:
col_names.keys()

dict_keys(['xna', 'xle', 'xhc', 'xhe', 'fna', 'fle', 'fhc', 'fhe', 'yld', 'Tray_temp', 'duty', 'Rate_m3', 'Rate_ton', 'sle', 'shc', 'she', 'case'])

# A

In [6]:
A_X = pd.read_csv('./data/phase_1/test_170.csv',index_col=0).sample(1)[['T10','T50','T90','N+A']]
A_X.head()

,T10,T50,T90,N+A
114,87.9,111.0,142.5,46.574


In [7]:
A_pred = A.predict(A_X)
print(A_pred.sum(axis=1))
A_pred

0    100.0
dtype: float64


,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,3.072219,1.894287,0.359189,3.93047,3.773474,5.573771,0.756088,6.373081,6.58954,14.729407,3.596306,5.846799,6.713968,8.26631,4.788936,3.131051,7.50037,4.781068,2.23161,0.781891,3.818849,0.719502,0.771812


# B

In [8]:
B_pred = B(A_pred)
print(B_pred.sum(axis=1))
B_pred.head()

0    100.0
dtype: float64


,O2,METHANE,ETHANE,PROPANE,BUTANE,IPENTANE,PENTANE,T2PENTEN,CP,3MP,T2HEXENE,HEXANE,MCP,22MP,BENZENE,CH,223MB,4M1HEXEN,2MHX,1T3M,HEPTANE,H2O,MCH,ECP,23DM1HX,TOLUENE,2MHEPT,OCTANE,ECH,EBENZENE,PXYLENE,MXYLENE,OXYLENE,2MOCTN,1NONENE,NONANE,CUMENE,1T35TMCH,PCH,PRBENZEN,135MBENZ,1M2EBNZN,123MBENZ,24DMOCT,TEMH,3MN,TBCYHXAN,DECANE,ODEBZ,UNDECANE,PNBZ,DODECANE,HXBNZN,TRIDECAN
0,0.0,0.0,0.0,0.094714,0.189429,1.572258,3.072219,0.037886,0.359189,3.698005,0.075469,3.93047,3.616021,0.631191,0.756088,1.95775,0.693548,0.013835,5.250967,1.363435,6.373081,0.0,7.897899,5.468073,0.06714,3.596306,6.646828,5.846799,8.26631,1.137157,0.645067,1.592189,1.414523,7.309841,0.190529,3.131051,0.031629,2.86851,1.912558,0.134912,0.579376,0.856252,0.629441,0.642191,0.952851,2.223807,0.719502,0.655672,0.721721,0.124008,0.041959,0.002211,0.008132,0.0


# B2 54 to 23

In [9]:
B.W.shape

(23, 54)

# C

In [10]:
case = pd.read_csv('./data/phase_2/cleaned/train.csv',index_col=0)[col_names['case']].sample(1).reset_index(drop=True)
C_X = case.join(B_pred)
C_X

,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_C7+ in Light End (Input)_wt%,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C6P- in Heart Cut (Input)_wt%,O2,METHANE,ETHANE,PROPANE,BUTANE,IPENTANE,PENTANE,T2PENTEN,CP,3MP,T2HEXENE,HEXANE,MCP,22MP,BENZENE,CH,223MB,4M1HEXEN,2MHX,1T3M,HEPTANE,H2O,MCH,ECP,23DM1HX,TOLUENE,2MHEPT,OCTANE,ECH,EBENZENE,PXYLENE,MXYLENE,OXYLENE,2MOCTN,1NONENE,NONANE,CUMENE,1T35TMCH,PCH,PRBENZEN,135MBENZ,1M2EBNZN,123MBENZ,24DMOCT,TEMH,3MN,TBCYHXAN,DECANE,ODEBZ,UNDECANE,PNBZ,DODECANE,HXBNZN,TRIDECAN
0,20.0,5.0,100.0,1.5,0.0,0.0,0.0,0.094714,0.189429,1.572258,3.072219,0.037886,0.359189,3.698005,0.075469,3.93047,3.616021,0.631191,0.756088,1.95775,0.693548,0.013835,5.250967,1.363435,6.373081,0.0,7.897899,5.468073,0.06714,3.596306,6.646828,5.846799,8.26631,1.137157,0.645067,1.592189,1.414523,7.309841,0.190529,3.131051,0.031629,2.86851,1.912558,0.134912,0.579376,0.856252,0.629441,0.642191,0.952851,2.223807,0.719502,0.655672,0.721721,0.124008,0.041959,0.002211,0.008132,0.0


In [11]:
C_pred = C.predict(C_X)
print(C_pred.sum(axis=1))
C_pred

0    54.0
dtype: float32


,Individual Component to Light End Split Factor_Oxygen_Ratio,Individual Component to Heart Cut Split Factor_Oxygen_Ratio,Individual Component to Heavy End Split Factor_Oxygen_Ratio,Individual Component to Light End Split Factor_Methane_Ratio,Individual Component to Heart Cut Split Factor_Methane_Ratio,Individual Component to Heavy End Split Factor_Methane_Ratio,Individual Component to Light End Split Factor_Ethane_Ratio,Individual Component to Heart Cut Split Factor_Ethane_Ratio,Individual Component to Heavy End Split Factor_Ethane_Ratio,Individual Component to Light End Split Factor_Propane_Ratio,Individual Component to Heart Cut Split Factor_Propane_Ratio,Individual Component to Heavy End Split Factor_Propane_Ratio,Individual Component to Light End Split Factor_n-Butane_Ratio,Individual Component to Heart Cut Split Factor_n-Butane_Ratio,Individual Component to Heavy End Split Factor_n-Butane_Ratio,Individual Component to Light End Split Factor_i-Pentane_Ratio,Individual Component to Heart Cut Split Factor_i-Pentane_Ratio,Individual Component to Heavy End Split Factor_i-Pentane_Ratio,Individual Component to Light End Split Factor_n-Pentane_Ratio,Individual Component to Heart Cut Split Factor_n-Pentane_Ratio,Individual Component to Heavy End Split Factor_n-Pentane_Ratio,Individual Component to Light End Split Factor_tr2-Pentene_Ratio,Individual Component to Heart Cut Split Factor_tr2-Pentene_Ratio,Individual Component to Heavy End Split Factor_tr2-Pentene_Ratio,Individual Component to Light End Split Factor_Cyclopentane_Ratio,Individual Component to Heart Cut Split Factor_Cyclopentane_Ratio,Individual Component to Heavy End Split Factor_Cyclopentane_Ratio,Individual Component to Light End Split Factor_3-Methylpentane_Ratio,Individual Component to Heart Cut Split Factor_3-Methylpentane_Ratio,Individual Component to Heavy End Split Factor_3-Methylpentane_Ratio,Individual Component to Light End Split Factor_tr2-Hexene_Ratio,Individual Component to Heart Cut Split Factor_tr2-Hexene_Ratio,Individual Component to Heavy End Split Factor_tr2-Hexene_Ratio,Individual Component to Light End Split Factor_n-Hexane_Ratio,Individual Component to Heart Cut Split Factor_n-Hexane_Ratio,Individual Component to Heavy End Split Factor_n-Hexane_Ratio,Individual Component to Light End Split Factor_Methylcyclopentane_Ratio,Individual Component to Heart Cut Split Factor_Methylcyclopentane_Ratio,Individual Component to Heavy End Split Factor_Methylcyclopentane_Ratio,"Individual Component to Light End Split Factor_2,2-Dimethylpentane_Ratio","Individual Component to Heart Cut Split Factor_2,2-Dimethylpentane_Ratio","Individual Component to Heavy End Split Factor_2,2-Dimethylpentane_Ratio",Individual Component to Light End Split Factor_Benzene_Ratio,Individual Component to Heart Cut Split Factor_Benzene_Ratio,Individual Component to Heavy End Split Factor_Benzene_Ratio,Individual Component to Light End Split Factor_Cyclohexane_Ratio,Individual Component to Heart Cut Split Factor_Cyclohexane_Ratio,Individual Component to Heavy End Split Factor_Cyclohexane_Ratio,"Individual Component to Light End Split Factor_2,2,3-Trimethylbutane_Ratio","Individual Component to Heart Cut Split Factor_2,2,3-Trimethylbutane_Ratio","Individual Component to Heavy End Split Factor_2,2,3-Trimethylbutane_Ratio",Individual Component to Light End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Heart Cut Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Heavy End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Light End Split Factor_2-Methylhexane_Ratio,Individual Component to Heart Cut Split Factor_2-Methylhexane_Ratio,Individual Component to Heavy End Split Factor_2-Methylhexane_Ratio,"Individual Component to Light End Split Factor_t-1,3-Dimethylcyclopentane_Ratio","Individual Component to Heart Cut Split Factor_t-1,3-Dimethylcyclopentane_Ratio","Individual Component to Heavy End Split Factor_t-1,3-Dimethylcyclopentane_Ratio",Individual Compone

# D

In [12]:
D_pred = D(B_pred,C_pred)
print(D_pred.sum(axis=1))
D_pred

0    300.0
dtype: float64


,Light End Product Properties_Oxygen_wt%,Light End Product Properties_Methane_wt%,Light End Product Properties_Ethane_wt%,Light End Product Properties_Propane_wt%,Light End Product Properties_n-Butane_wt%,Light End Product Properties_i-Pentane_wt%,Light End Product Properties_n-Pentane_wt%,Light End Product Properties_tr2-Pentene_wt%,Light End Product Properties_Cyclopentane_wt%,Light End Product Properties_3-Methylpentane_wt%,Light End Product Properties_tr2-Hexene_wt%,Light End Product Properties_n-Hexane_wt%,Light End Product Properties_Methylcyclopentane_wt%,"Light End Product Properties_2,2-Dimethylpentane_wt%",Light End Product Properties_Benzene_wt%,Light End Product Properties_Cyclohexane_wt%,"Light End Product Properties_2,2,3-Trimethylbutane_wt%",Light End Product Properties_4-Methyl-1-hexene_wt%,Light End Product Properties_2-Methylhexane_wt%,"Light End Product Properties_t-1,3-Dimethylcyclopentane_wt%",Light End Product Properties_n-Heptane_wt%,Light End Product Properties_H2O_wt%,Light End Product Properties_Methylcyclohexane_wt%,Light End Product Properties_Ethylcyclopentane_wt%,"Light End Product Properties_2,3-Dimethyl-1-hexene_wt%",Light End Product Properties_Toluene_wt%,Light End Product Properties_2-Methylheptane_wt%,Light End Product Properties_n-Octane_wt%,Light End Product Properties_Ethylcyclohexane_wt%,Light End Product Properties_EthylBenzene_wt%,Light End Product Properties_P-Xylene_wt%,Light End Product Properties_M-Xylene_wt%,Light End Product Properties_O-Xylene_wt%,Light End Product Properties_2-Methyloctane_wt%,Light End Product Properties_1-Nonene_wt%,Light End Product Properties_n-Nonane_wt%,Light End Product Properties_i-Propylbenzene_wt%,"Light End Product Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Light End Product Properties_n-Propylcyclohexane_wt%,Light End Product Properties_n-Propylbenzene_wt%,"Light End Product Properties_1,3,5-Trimethylbenzene_wt%",Light End Product Properties_1-Methyl-2-ethylbenzene_wt%,"Light End Product Properties_1,2,3-Trimethylbenzene_wt%","Light End Product Properties_2,4-Dimethyloctane_wt%","Light End Product Properties_2,2,3,3-Tetramethylhexane_wt%",Light End Product Properties_3-Methylnonane_wt%,Light End Product Properties_tert-Butylcyclohexane_wt%,Light End Product Properties_n-Decane_wt%,"Light End Product Properties_1,2-Diethylbenzene_wt%",Light End Product Properties_n-Undecane_wt%,Light End Product Properties_n-Pentylbenzene_wt%,Light End Product Properties_n-Dodecane_wt%,Light End Product Properties_n-Hexylbenzene_wt%,Light End Product Properties_n-Tridecane_wt%,Heart Cut Product Properties_Oxygen_wt%,Heart Cut Product Properties_Methane_wt%,Heart Cut Product Properties_Ethane_wt%,Heart Cut Product Properties_Propane_wt%,Heart Cut Product Properties_n-Butane_wt%,Heart Cut Product Properties_i-Pentane_wt%,Heart Cut Product Properties_n-Pentane_wt%,Heart Cut Product Properties_tr2-Pentene_wt%,Heart Cut Product Properties_Cyclopentane_wt%,Heart Cut Product Properties_3-Methylpentane_wt%,Heart Cut Product Properties_tr2-Hexene_wt%,Heart Cut Product Properties_n-Hexane_wt%,Heart Cut Product Properties_Methylcyclopentane_wt%,"Heart Cut Product Properties_2,2-Dimethylpentane_wt%",Heart Cut Product Properties_Benzene_wt%,Heart Cut Product Properties_Cyclohexane_wt%,"Heart Cut Product Properties_2,2,3-Trimethylbutane_wt%",Heart Cut Product Properties_4-Methyl-1-hexene_wt%,Heart Cut Product Properties_2-Methylhexane_wt%,"Heart Cut Product Properties_t-1,3-Dimethylcyclopentane_wt%",Heart Cut Product Properties_n-Heptane_wt%,Heart Cut Product Properties_H2O_wt%,Heart Cut Product Properties_Methylcyclohexane_wt%,Heart Cut Product Properties_Ethylcyclopentane_wt%,"Heart Cut Product Properties_2,3-Dimethyl-1-hexene_wt%",Heart Cut Product Properties_Toluene_wt%,Heart Cut Product Properties_2-Methylheptane_wt%,Heart Cut Product Properties_n-Octane_wt%,Heart Cut Product Properties_Ethylcyclohexane_wt%,Heart Cut Product Properties_EthylBenzene_wt%,Heart Cut Product

# E

In [13]:
df = pd.read_csv('./data/phase_2/cleaned/train(energy).csv',index_col=0).dropna(axis=0).sample(1)
E_X = df[E.x_col]
E_X

,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_C7+ in Light End (Input)_wt%,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C6P- in Heart Cut (Input)_wt%,Naphtha Properties_Oxygen_wt%,Naphtha Properties_Methane_wt%,Naphtha Properties_Ethane_wt%,Naphtha Properties_Propane_wt%,Naphtha Properties_n-Butane_wt%,Naphtha Properties_i-Pentane_wt%,Naphtha Properties_n-Pentane_wt%,Naphtha Properties_tr2-Pentene_wt%,Naphtha Properties_Cyclopentane_wt%,Naphtha Properties_3-Methylpentane_wt%,Naphtha Properties_tr2-Hexene_wt%,Naphtha Properties_n-Hexane_wt%,Naphtha Properties_Methylcyclopentane_wt%,"Naphtha Properties_2,2-Dimethylpentane_wt%",Naphtha Properties_Benzene_wt%,Naphtha Properties_Cyclohexane_wt%,"Naphtha Properties_2,2,3-Trimethylbutane_wt%",Naphtha Properties_4-Methyl-1-hexene_wt%,Naphtha Properties_2-Methylhexane_wt%,"Naphtha Properties_t-1,3-Dimethylcyclopentane_wt%",Naphtha Properties_n-Heptane_wt%,Naphtha Properties_H2O_wt%,Naphtha Properties_Methylcyclohexane_wt%,Naphtha Properties_Ethylcyclopentane_wt%,"Naphtha Properties_2,3-Dimethyl-1-hexene_wt%",Naphtha Properties_Toluene_wt%,Naphtha Properties_2-Methylheptane_wt%,Naphtha Properties_n-Octane_wt%,Naphtha Properties_Ethylcyclohexane_wt%,Naphtha Properties_EthylBenzene_wt%,Naphtha Properties_P-Xylene_wt%,Naphtha Properties_M-Xylene_wt%,Naphtha Properties_O-Xylene_wt%,Naphtha Properties_2-Methyloctane_wt%,Naphtha Properties_1-Nonene_wt%,Naphtha Properties_n-Nonane_wt%,Naphtha Properties_i-Propylbenzene_wt%,"Naphtha Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Naphtha Properties_n-Propylcyclohexane_wt%,Naphtha Properties_n-Propylbenzene_wt%,"Naphtha Properties_1,3,5-Trimethylbenzene_wt%",Naphtha Properties_1-Methyl-2-ethylbenzene_wt%,"Naphtha Properties_1,2,3-Trimethylbenzene_wt%","Naphtha Properties_2,4-Dimethyloctane_wt%","Naphtha Properties_2,2,3,3-Tetramethylhexane_wt%",Naphtha Properties_3-Methylnonane_wt%,Naphtha Properties_tert-Butylcyclohexane_wt%,Naphtha Properties_n-Decane_wt%,"Naphtha Properties_1,2-Diethylbenzene_wt%",Naphtha Properties_n-Undecane_wt%,Naphtha Properties_n-Pentylbenzene_wt%,Naphtha Properties_n-Dodecane_wt%,Naphtha Properties_n-Hexylbenzene_wt%,Naphtha Properties_n-Tridecane_wt%
407-02,10.0,5.0,100.0,1.0,0.0,0.0,0.0,0.079997,0.159994,1.328947,2.224911,0.031999,0.358986,3.335867,0.067997,3.581856,4.166833,0.459982,0.439982,2.25591,0.50598,0.01,3.828847,1.724931,5.583776,0.0,9.9916,6.917723,0.022999,3.25087,2.24191,5.618775,13.707452,0.932963,0.528979,1.305948,1.159954,4.913803,0.127995,2.813887,0.033999,3.736851,2.4919,0.144994,0.621975,0.918963,0.674973,0.73997,1.097956,2.562897,0.873965,1.148954,0.981961,0.216991,0.056998,0.004,0.011,0.0


In [14]:
E_pred = E.predict(E_X)
E_pred

,Duty_E062 3.5K Steam Rate_ton/hr,Duty_Reboiler Heat Duty_Mkcal/hr,NA_density,LE_density,HC_density,HE_density
0,2.506068,5.019242,0.747316,0.681595,0.75901,0.775698


# F

In [15]:
F_X = D_pred[col_names['xhc']]
F_X

,Heart Cut Product Properties_Oxygen_wt%,Heart Cut Product Properties_Methane_wt%,Heart Cut Product Properties_Ethane_wt%,Heart Cut Product Properties_Propane_wt%,Heart Cut Product Properties_n-Butane_wt%,Heart Cut Product Properties_i-Pentane_wt%,Heart Cut Product Properties_n-Pentane_wt%,Heart Cut Product Properties_tr2-Pentene_wt%,Heart Cut Product Properties_Cyclopentane_wt%,Heart Cut Product Properties_3-Methylpentane_wt%,Heart Cut Product Properties_tr2-Hexene_wt%,Heart Cut Product Properties_n-Hexane_wt%,Heart Cut Product Properties_Methylcyclopentane_wt%,"Heart Cut Product Properties_2,2-Dimethylpentane_wt%",Heart Cut Product Properties_Benzene_wt%,Heart Cut Product Properties_Cyclohexane_wt%,"Heart Cut Product Properties_2,2,3-Trimethylbutane_wt%",Heart Cut Product Properties_4-Methyl-1-hexene_wt%,Heart Cut Product Properties_2-Methylhexane_wt%,"Heart Cut Product Properties_t-1,3-Dimethylcyclopentane_wt%",Heart Cut Product Properties_n-Heptane_wt%,Heart Cut Product Properties_H2O_wt%,Heart Cut Product Properties_Methylcyclohexane_wt%,Heart Cut Product Properties_Ethylcyclopentane_wt%,"Heart Cut Product Properties_2,3-Dimethyl-1-hexene_wt%",Heart Cut Product Properties_Toluene_wt%,Heart Cut Product Properties_2-Methylheptane_wt%,Heart Cut Product Properties_n-Octane_wt%,Heart Cut Product Properties_Ethylcyclohexane_wt%,Heart Cut Product Properties_EthylBenzene_wt%,Heart Cut Product Properties_P-Xylene_wt%,Heart Cut Product Properties_M-Xylene_wt%,Heart Cut Product Properties_O-Xylene_wt%,Heart Cut Product Properties_2-Methyloctane_wt%,Heart Cut Product Properties_1-Nonene_wt%,Heart Cut Product Properties_n-Nonane_wt%,Heart Cut Product Properties_i-Propylbenzene_wt%,"Heart Cut Product Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Heart Cut Product Properties_n-Propylcyclohexane_wt%,Heart Cut Product Properties_n-Propylbenzene_wt%,"Heart Cut Product Properties_1,3,5-Trimethylbenzene_wt%",Heart Cut Product Properties_1-Methyl-2-ethylbenzene_wt%,"Heart Cut Product Properties_1,2,3-Trimethylbenzene_wt%","Heart Cut Product Properties_2,4-Dimethyloctane_wt%","Heart Cut Product Properties_2,2,3,3-Tetramethylhexane_wt%",Heart Cut Product Properties_3-Methylnonane_wt%,Heart Cut Product Properties_tert-Butylcyclohexane_wt%,Heart Cut Product Properties_n-Decane_wt%,"Heart Cut Product Properties_1,2-Diethylbenzene_wt%",Heart Cut Product Properties_n-Undecane_wt%,Heart Cut Product Properties_n-Pentylbenzene_wt%,Heart Cut Product Properties_n-Dodecane_wt%,Heart Cut Product Properties_n-Hexylbenzene_wt%,Heart Cut Product Properties_n-Tridecane_wt%
0,0.0,0.0,0.0,0.000872,0.004022,0.06411,0.147725,0.001709,0.027372,0.456795,0.020294,0.77699,1.473265,0.42128,0.337822,1.422655,0.580936,0.015855,6.590658,1.757768,8.334864,0.0,10.346765,7.163459,0.087978,4.711504,8.710157,7.659926,10.771553,1.481774,0.834754,2.048288,1.677735,8.012323,0.180755,2.732236,0.027231,3.294762,1.544441,0.1082,0.443588,0.692134,0.461887,0.528683,0.744807,1.660434,0.533488,0.474441,0.513186,0.083809,0.027963,0.001455,0.005293,0.0


In [16]:
F_pred = F(F_X)
print(F_pred.sum(axis=1))
F_pred

0    100.0
dtype: float64


,Hydrogen,Methane,Ethane,Propane,n-Butane,i-Butane,C4 Naphthenes,n-Pentane,C5 i-Paraffins,C5 Naphthenes,n-Hexane,C6 i-Paraffins,C6 Naphthenes,Benzene,n-Heptane,C7 i-Paraffins,C7 Naphthenes,Toluene,n-Octane,C8 i-Paraffins,C8 Naphthenes,Ethylbenzene,P-Xylene,M-Xylene,O-Xylene,n-Nonane,C9 i-Paraffins,C9 Naphthenes,C9 Aromtics,C10+ n-Paraffins,C10+ i-Paraffins,C10+ Naphthenes,C10+ Aromtics
0,0.0,0.0,0.0,0.000872,0.001609,0.002212,0.000201,0.147725,0.065819,0.027372,0.77699,0.477089,2.89592,0.337822,8.334864,7.608728,19.267993,4.711504,7.659926,8.798135,10.771553,1.481774,0.834754,2.048288,1.677735,2.732236,8.193077,4.839203,1.733041,0.559705,2.933925,0.533488,0.546442


# G

In [17]:
G_pred = G(F_pred)
print(G_pred.sum(axis=1))
G_pred

0    100.0
dtype: float64


,HYD,NC1,NC2,NC3,PC4,SAP,SAN,BEZ,TOL,ETB,PXY,MXY,OXY,C9A,C10
0,0.0,0.0,0.0,0.000872,0.003821,48.28842,38.335528,0.337822,4.711504,1.481774,0.834754,2.048288,1.677735,1.733041,0.546442


# H

In [18]:
x = np.random.uniform(size=(1,len(H.x_col)))
x = pd.DataFrame(x,columns=H.x_col)
x

,"N+2A in Naphtha Feed, wt%","Chlorine in Fresh Catalyst, wt%","Carbon in Spent Catalyst, %","C6P- in Naphtha Feed, wt%","3R211 Inlet Tem, oC","3R212 Inlet Temp, oC","3R213 Inlet Temp, oC","3R214 Inlet Temp, oC",Feed Hydrogen,Feed Methane,Feed Ethane,Feed Propane,Feed n-Butane,Feed i-Butane,Feed C4 Naphthenes,Feed n-Pentane,Feed C5 i-Paraffins,Feed C5 Naphthenes,Feed n-Hexane,Feed C6 i-Paraffins,Feed C6 Naphthenes,Feed Benzene,Feed n-Heptane,Feed C7 i-Paraffins,Feed C7 Naphthenes,Feed Toluene,Feed n-Octane,Feed C8 i-Paraffins,Feed C8 Naphthenes,Feed Ethylbenzene,Feed P-Xylene,Feed M-Xylene,Feed O-Xylene,Feed n-Nonane,Feed C9 i-Paraffins,Feed C9 Naphthenes,Feed C9 Aromtics,Feed C10+ n-Paraffins,Feed C10+ i-Paraffins,Feed C10+ Naphthenes,Feed C10+ Aromtics,"Product Separator Pressure, kg/cm2_g",H2/HC Ratio
0,0.44837,0.092717,0.207146,0.482789,0.359764,0.252649,0.59464,0.080147,0.664898,0.618456,0.744922,0.871027,0.048754,0.665681,0.399075,0.533447,0.271722,0.687926,0.956973,0.896798,0.370071,0.365891,0.970725,0.46881,0.199492,0.269795,0.979477,0.31415,0.633137,0.100738,0.23396,0.727949,0.730732,0.582283,0.692603,0.612441,0.673695,0.096599,0.555153,0.698986,0.742234,0.837192,0.953263


In [19]:
H.predict(x)

,Product Hydrogen,Product Methane,Product Ethane,Product Propane,Product n-Butane,Product i-Butane,Product C4 Naphthenes,Product n-Pentane,Product C5 i-Paraffins,Product C5 Naphthenes,Product n-Hexane,Product C6 i-Paraffins,Product C6 Naphthenes,Product Benzene,Product n-Heptane,Product C7 i-Paraffins,Product C7 Naphthenes,Product Toluene,Product n-Octane,Product C8 i-Paraffins,Product C8 Naphthenes,Product Ethylbenzene,Product P-Xylene,Product M-Xylene,Product O-Xylene,Product n-Nonane,Product C9 i-Paraffins,Product C9 Naphthenes,Product C9 Aromtics,Product C10+ n-Paraffins,Product C10+ i-Paraffins,Product C10+ Naphthenes,Product C10+ Aromtics
0,0.915389,0.183666,1.050885,0.4378,0.141711,0.123236,0.0,0.284176,0.446832,0.002217,0.142793,0.352293,0.014012,0.388875,0.413995,1.452981,0.021304,3.148664,0.140866,0.082615,0.019279,0.802828,0.503716,3.391492,1.244689,0.000969,0.004213,0.000356,2.371929,0.002622,0.006843,0.004299,0.281054


# I

In [20]:
A_pred = A.predict(A_X)
print(A_pred.sum(axis=1))
A_pred

0    100.0
dtype: float64


,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,3.072219,1.894287,0.359189,3.93047,3.773474,5.573771,0.756088,6.373081,6.58954,14.729407,3.596306,5.846799,6.713968,8.26631,4.788936,3.131051,7.50037,4.781068,2.23161,0.781891,3.818849,0.719502,0.771812


In [21]:
B_pred = B(A_pred)
print(B_pred.sum(axis=1))
B_pred.head()

0    100.0
dtype: float64


,O2,METHANE,ETHANE,PROPANE,BUTANE,IPENTANE,PENTANE,T2PENTEN,CP,3MP,T2HEXENE,HEXANE,MCP,22MP,BENZENE,CH,223MB,4M1HEXEN,2MHX,1T3M,HEPTANE,H2O,MCH,ECP,23DM1HX,TOLUENE,2MHEPT,OCTANE,ECH,EBENZENE,PXYLENE,MXYLENE,OXYLENE,2MOCTN,1NONENE,NONANE,CUMENE,1T35TMCH,PCH,PRBENZEN,135MBENZ,1M2EBNZN,123MBENZ,24DMOCT,TEMH,3MN,TBCYHXAN,DECANE,ODEBZ,UNDECANE,PNBZ,DODECANE,HXBNZN,TRIDECAN
0,0.0,0.0,0.0,0.094714,0.189429,1.572258,3.072219,0.037886,0.359189,3.698005,0.075469,3.93047,3.616021,0.631191,0.756088,1.95775,0.693548,0.013835,5.250967,1.363435,6.373081,0.0,7.897899,5.468073,0.06714,3.596306,6.646828,5.846799,8.26631,1.137157,0.645067,1.592189,1.414523,7.309841,0.190529,3.131051,0.031629,2.86851,1.912558,0.134912,0.579376,0.856252,0.629441,0.642191,0.952851,2.223807,0.719502,0.655672,0.721721,0.124008,0.041959,0.002211,0.008132,0.0


In [22]:
I(B_pred)

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,3.072219,1.894287,0.359189,3.93047,3.773474,5.573771,0.756088,6.373081,6.58954,14.729407,3.596306,5.846799,6.713968,8.26631,4.788936,3.131051,7.50037,4.781068,2.23161,0.781891,3.818849,0.719502,0.771812


# 把全部模組整合到一塊

In [23]:
class EVA(object):
    def __init__(self):
        self.A = joblib.load('./model/SVR(4_to_23).pkl')
        self.B = joblib.load('./model/transformer(23_to_54).pkl')
        self.C = joblib.load('./model/ANN(58_to_sp162).pkl')
        self.D = joblib.load('./model/transformer(SP162_to_Y162).pkl')
        self.E = joblib.load('./model/ANN(energy).pkl')
        self.F = joblib.load('./model/transformer(54_to_33).pkl')
        self.G = joblib.load('./model/transformer(33_to_15).pkl')
        self.H = joblib.load('./model/transformer(43_to_33).pkl')
        self.I = joblib.load('./model/transformer(54_to_23).pkl')
        self.col_names = joblib.load('./data/phase_2/cleaned/col_names.pkl')

    def __call__(self, X):
        x4, case4 = X.iloc[:, :4], X.iloc[:, 4:]
        # model forward
        self.y23 = self.A.predict(x4)  # percent sum = 100
        self.Xna = self.B(self.y23)  # percent sum = 100
        # percent sum = 1 * 54component = 54
        self.sp162 = self.C.predict(case4.join(self.Xna))
        # percent sum = 100*3(le,hc,he) = 300
        self.y162 = self.D(self.Xna, self.sp162)
        self.xhc = self.y162[self.col_names['xhc']]  # percent sum = 100
        self.xhc33_p = self.F(self.xhc)/100  # percent

        # predict duty and density(ton/m3)
        E_pred = self.E.predict(case4.join(self.Xna))
        self.duty = E_pred.iloc[:, :2]
        self.density = E_pred.iloc[:, 2:]

        # get split factor
        self.shc = self.sp162[self.col_names['shc']]
        self.sle = self.sp162[self.col_names['sle']]
        self.she = self.sp162[self.col_names['she']]

        # fhc_ton = fhc_m3*density
        self.fhc_m3 = case4['Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr']
        self.fhc_ton = self.fhc_m3.values * self.density.iloc[:, 2].values

        # calculate fle_ton,fhe_ton
        self.fle_ton = self.fhc_ton * ((self.Xna.values@self.sle.values.ravel()) /
                                       (self.Xna.values@self.shc.values.ravel()))
        self.fhe_ton = self.fhc_ton * ((self.Xna.values@self.she.values.ravel()) /
                                       (self.Xna.values@self.shc.values.ravel()))

        # calculate fna
        self.fna_ton = self.fle_ton + self.fhc_ton + self.fhe_ton

        # scale fle,fhc,fhe
        self.fle_ton = self.fle_ton*100000/self.fna_ton
        self.fhc_ton = self.fhc_ton*100000/self.fna_ton
        self.fhe_ton = self.fhe_ton*100000/self.fna_ton

        # add columns name
        self.fle_ton = pd.Series(
            self.fle_ton, name=self.col_names['Rate_ton'][1])
        self.fhe_ton = pd.Series(
            self.fhe_ton, name=self.col_names['Rate_ton'][3])

        # scale duty
        self.duty *= 100/self.fna_ton[0]

        # xhc33 = fhc_ton*xhc_33_p
        self.xhc33 = self.fhc_ton[0]*self.xhc33_p
        self.xhc33_tmp = self.xhc33*74000/self.fhc_ton[0]

        # xhe33 = H(someting + xhc33 + something)
        xhc23 = self.I(self.xhc)
        C6Pm = xhc23[['C5NP', 'C5IP', 'C6IP', 'C6NP']].sum(axis=1)[0]

        NpA = xhc23[['C5N', 'C6N', 'C7N', 'C8N', 'C9N', 'C10N']].sum(axis=1)[0] +\
            xhc23[['C6A', 'C7A', 'C8A', 'C9A', 'C10A']].sum(axis=1)[0]*2

        self.H_input = [NpA, 0.942, 3.78, C6Pm, 517, 517, 517, 517] + \
            self.xhc33_tmp.values.ravel().tolist()+[4.798, 1.44]
        self.H_input = np.array([self.H_input])
        self.H_input = pd.DataFrame(self.H_input, columns=self.H.x_col)
        self.reformer_input = [NpA, 0.942, 3.78, C6Pm, 517, 517, 517, 517] + self.xhc33.values.ravel().tolist() + [4.798, 1.44]
        self.reformer_input = np.array([self.reformer_input])
        self.reformer_input = pd.DataFrame(self.reformer_input, columns=self.H.x_col)

        self.重組33_tmp = self.H.predict(self.H_input)

        self.重組33 = self.重組33_tmp *self.fhc_ton[0]/74000

        # final output
        self.重組15 = self.G(self.重組33)

        # raname and combine return to user
        self.predict = self.重組15.join(self.fle_ton).join(
            self.fhe_ton).join(self.duty)
        self.naphtha = self.y23
        self.pre_d = self.sp162
        self.reform = self.重組33

In [24]:
eva = EVA()

# save eva

In [25]:
joblib.dump(eva,'./model/eva.pkl')
del eva

# load eva

In [26]:
eva = joblib.load('./model/eva.pkl')

# create input X for eva

In [27]:
X = pd.DataFrame(np.hstack((A_X.values,case.values)))
X.columns = A_X.columns.tolist()+case.columns.tolist()
X.head()

,T10,T50,T90,N+A,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_C7+ in Light End (Input)_wt%,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C6P- in Heart Cut (Input)_wt%
0,87.9,111.0,142.5,46.574,20.0,5.0,100.0,1.5


# get eva result

In [28]:
eva(X)

In [29]:
eva.predict

,HYD,NC1,NC2,NC3,PC4,SAP,SAN,BEZ,TOL,ETB,PXY,MXY,OXY,C9A,C10,Light End Product Properties_Light End Prod. Rate_ton/hr,Heavy End Product Properties_Heavy End Product Rate_ton/hr,Duty_E062 3.5K Steam Rate_ton/hr,Duty_Reboiler Heat Duty_Mkcal/hr
0,2657.781622,865.495784,2215.029405,2474.089081,2104.561838,13640.572597,652.706843,2710.499459,14882.359351,3497.812108,2284.227027,10467.187892,5260.547027,10662.04973,1908.139676,16586.349933,7130.588254,2.607818,5.385278


In [30]:
eva.xhc33

,Hydrogen,Methane,Ethane,Propane,n-Butane,i-Butane,C4 Naphthenes,n-Pentane,C5 i-Paraffins,C5 Naphthenes,n-Hexane,C6 i-Paraffins,C6 Naphthenes,Benzene,n-Heptane,C7 i-Paraffins,C7 Naphthenes,Toluene,n-Octane,C8 i-Paraffins,C8 Naphthenes,Ethylbenzene,P-Xylene,M-Xylene,O-Xylene,n-Nonane,C9 i-Paraffins,C9 Naphthenes,C9 Aromtics,C10+ n-Paraffins,C10+ i-Paraffins,C10+ Naphthenes,C10+ Aromtics
0,0.0,0.0,0.0,0.665435,1.227253,1.687473,0.153407,112.688832,50.208884,20.879841,592.71167,363.938137,2209.096363,257.700862,6358.089467,5804.171035,14698.214797,3594.079367,5843.226212,6711.486473,8216.870518,1130.342218,636.775969,1562.496738,1279.827518,2084.232949,6249.930063,3691.492461,1322.016478,426.960138,2238.087749,406.960691,416.842812


In [31]:
eva.naphtha

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,3.072219,1.894287,0.359189,3.93047,3.773474,5.573771,0.756088,6.373081,6.58954,14.729407,3.596306,5.846799,6.713968,8.26631,4.788936,3.131051,7.50037,4.781068,2.23161,0.781891,3.818849,0.719502,0.771812


In [32]:
eva.pre_d

,Individual Component to Light End Split Factor_Oxygen_Ratio,Individual Component to Heart Cut Split Factor_Oxygen_Ratio,Individual Component to Heavy End Split Factor_Oxygen_Ratio,Individual Component to Light End Split Factor_Methane_Ratio,Individual Component to Heart Cut Split Factor_Methane_Ratio,Individual Component to Heavy End Split Factor_Methane_Ratio,Individual Component to Light End Split Factor_Ethane_Ratio,Individual Component to Heart Cut Split Factor_Ethane_Ratio,Individual Component to Heavy End Split Factor_Ethane_Ratio,Individual Component to Light End Split Factor_Propane_Ratio,Individual Component to Heart Cut Split Factor_Propane_Ratio,Individual Component to Heavy End Split Factor_Propane_Ratio,Individual Component to Light End Split Factor_n-Butane_Ratio,Individual Component to Heart Cut Split Factor_n-Butane_Ratio,Individual Component to Heavy End Split Factor_n-Butane_Ratio,Individual Component to Light End Split Factor_i-Pentane_Ratio,Individual Component to Heart Cut Split Factor_i-Pentane_Ratio,Individual Component to Heavy End Split Factor_i-Pentane_Ratio,Individual Component to Light End Split Factor_n-Pentane_Ratio,Individual Component to Heart Cut Split Factor_n-Pentane_Ratio,Individual Component to Heavy End Split Factor_n-Pentane_Ratio,Individual Component to Light End Split Factor_tr2-Pentene_Ratio,Individual Component to Heart Cut Split Factor_tr2-Pentene_Ratio,Individual Component to Heavy End Split Factor_tr2-Pentene_Ratio,Individual Component to Light End Split Factor_Cyclopentane_Ratio,Individual Component to Heart Cut Split Factor_Cyclopentane_Ratio,Individual Component to Heavy End Split Factor_Cyclopentane_Ratio,Individual Component to Light End Split Factor_3-Methylpentane_Ratio,Individual Component to Heart Cut Split Factor_3-Methylpentane_Ratio,Individual Component to Heavy End Split Factor_3-Methylpentane_Ratio,Individual Component to Light End Split Factor_tr2-Hexene_Ratio,Individual Component to Heart Cut Split Factor_tr2-Hexene_Ratio,Individual Component to Heavy End Split Factor_tr2-Hexene_Ratio,Individual Component to Light End Split Factor_n-Hexane_Ratio,Individual Component to Heart Cut Split Factor_n-Hexane_Ratio,Individual Component to Heavy End Split Factor_n-Hexane_Ratio,Individual Component to Light End Split Factor_Methylcyclopentane_Ratio,Individual Component to Heart Cut Split Factor_Methylcyclopentane_Ratio,Individual Component to Heavy End Split Factor_Methylcyclopentane_Ratio,"Individual Component to Light End Split Factor_2,2-Dimethylpentane_Ratio","Individual Component to Heart Cut Split Factor_2,2-Dimethylpentane_Ratio","Individual Component to Heavy End Split Factor_2,2-Dimethylpentane_Ratio",Individual Component to Light End Split Factor_Benzene_Ratio,Individual Component to Heart Cut Split Factor_Benzene_Ratio,Individual Component to Heavy End Split Factor_Benzene_Ratio,Individual Component to Light End Split Factor_Cyclohexane_Ratio,Individual Component to Heart Cut Split Factor_Cyclohexane_Ratio,Individual Component to Heavy End Split Factor_Cyclohexane_Ratio,"Individual Component to Light End Split Factor_2,2,3-Trimethylbutane_Ratio","Individual Component to Heart Cut Split Factor_2,2,3-Trimethylbutane_Ratio","Individual Component to Heavy End Split Factor_2,2,3-Trimethylbutane_Ratio",Individual Component to Light End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Heart Cut Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Heavy End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Light End Split Factor_2-Methylhexane_Ratio,Individual Component to Heart Cut Split Factor_2-Methylhexane_Ratio,Individual Component to Heavy End Split Factor_2-Methylhexane_Ratio,"Individual Component to Light End Split Factor_t-1,3-Dimethylcyclopentane_Ratio","Individual Component to Heart Cut Split Factor_t-1,3-Dimethylcyclopentane_Ratio","Individual Component to Heavy End Split Factor_t-1,3-Dimethylcyclopentane_Ratio",Individual Compone

In [33]:
eva.reform

,Product Hydrogen,Product Methane,Product Ethane,Product Propane,Product n-Butane,Product i-Butane,Product C4 Naphthenes,Product n-Pentane,Product C5 i-Paraffins,Product C5 Naphthenes,Product n-Hexane,Product C6 i-Paraffins,Product C6 Naphthenes,Product Benzene,Product n-Heptane,Product C7 i-Paraffins,Product C7 Naphthenes,Product Toluene,Product n-Octane,Product C8 i-Paraffins,Product C8 Naphthenes,Product Ethylbenzene,Product P-Xylene,Product M-Xylene,Product O-Xylene,Product n-Nonane,Product C9 i-Paraffins,Product C9 Naphthenes,Product C9 Aromtics,Product C10+ n-Paraffins,Product C10+ i-Paraffins,Product C10+ Naphthenes,Product C10+ Aromtics
0,2657.781622,865.495784,2215.029405,2474.089081,1042.831351,1061.730486,1.453168,2020.660541,3063.023351,49.937483,813.515351,1884.579676,120.025081,2710.499459,948.364216,3629.209946,176.380514,14882.359351,219.530716,681.281243,172.724662,3497.812108,2284.227027,10467.187892,5260.547027,17.013686,103.236912,16.554819,10662.04973,92.118196,166.585595,117.084284,1908.139676
